# Preprocessing
Data set: wisc_bc_data_preprocessed
Source: Breast Cancer Wisconsin (Diagnostic) dataset
https://github.com/dataspelunking/MLwR/blob/master/Machine%20Learning%20with%20R%20(2nd%20Ed.)/Chapter%2003/wisc_bc_data.csv

1. Deleted the first column, id, which is not useful.
2. Moved the column of diagnosis to the last, as the column of label.
3. Normalized the data set using the mean and the standard deviation in each column.

## Part 1 Different values of k

Each experiment has five variable in total.
k value: variable
weights according to distance: true or false
threshold: variable or false
tie: true or false
splitting ratio: variable
where threshold comes together with weights according to distance.

In the following experiment, the ratio of training subset and testing subset is fixed to be 469:100 and the test cases are selected randomly.
In the first experiment, every nearest k-th neigbors gives one vote, irregardless of their distance.
In case of a tie, the first response is returned.
No threshold is added.

In [15]:
import csv
import random
import math
import operator
 
def loadDataset(filename, NumOfTest, trainingSet=[] , testSet=[]):
	with open(filename, 'r') as csvfile:
	    lines = csv.reader(csvfile)
	    next(lines, None)  #skip the first line which is headers
	    datarow = list(lines)
	    for x in range(len(datarow)):
	        for y in range(30):
	            datarow[x][y] = float(datarow[x][y])
	        if len(testSet)<NumOfTest:
	            if random.random() < 0.5:  #0<=random.random()<=1, 50% to be testSet
	                trainingSet.append(datarow[x])
	            else:
	                testSet.append(datarow[x])
	        else:
	            trainingSet.append(datarow[x])

def euclideanDistance(value1, value2, length):
	distance = 0
	for x in range(length):
		distance += pow((value1[x] - value2[x]), 2)
	return math.sqrt(distance)
 
def getNeighbors(trainingSet, testInstance, k):
	distances = []
	length = len(testInstance)-1
#last col:label
	for x in range(len(trainingSet)):
		dist = euclideanDistance(testInstance, trainingSet[x], length)
		distances.append([trainingSet[x], dist])
	distances.sort(key=operator.itemgetter(1))  #sort the first item in a list of distances list
	neighbors = []
	for x in range(k):
		neighbors.append(distances[x][0])  #append both trainingSet[x] and dist for k neighbors
	return neighbors
 
def getResponse(neighbors):
	classVotes = {}
	for x in range(len(neighbors)):
		response = neighbors[x][-1]
#-1: from the right
		if response in classVotes:
			classVotes[response] += 1
		else:
			classVotes[response] = 1
	sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
	return sortedVotes[0][0]
#return the first response in case of a tie

def getAccuracy(testSet, predictions):
	correct = 0
	for x in range(len(testSet)):
		if testSet[x][-1] == predictions[x]:
			correct += 1
	return (correct/float(len(testSet))) * 100.0
	
def main():
	# prepare data
	trainingSet=[]
	testSet=[]
	NumOfTest = 100
	loadDataset('wisc_bc_data_preprocessed.csv', NumOfTest, trainingSet, testSet)
	print ('Number of train cases: ' + repr(len(trainingSet)) )
	print ('Number of test cases: ' + repr(len(testSet)) )
	# generate predictions
	predictions=[]
	k = 7
	for x in range(len(testSet)):
		neighbors = getNeighbors(trainingSet, testSet[x], k)
		result = getResponse(neighbors)
		predictions.append(result)
		#print('predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))
	accuracy = getAccuracy(testSet, predictions)
	print('Accuracy: ' + repr(accuracy) + '%')
	
main()

Number of train cases: 469
Number of test cases: 100
Accuracy: 96.0%


when k=22, which is the square root(the number of train cases), the highest accuracy is propably:

Number of train cases: 469
Number of test cases: 100
Accuracy: 96.0%

when k=10, the highest accuracy increases apparently:
Number of train cases: 469
Number of test cases: 100
Accuracy: 98.0%

when k=7, the highest accuracy drops apparently:
Number of train cases: 469
Number of test cases: 100
Accuracy: 97.0%

## Part 2: Different ways to do the voting(where k>2)

## Part 2.1: Adding weights to the votes proportional to distance

In the this experiment, weights are added according to distance.
Assume k=10, which apparently gives a higher accuracy in part 1.
In case of a tie, the first response is returned.
No threshold is added.

In [16]:
def getNeighbors(trainingSet, testInstance, k):
	distances = []
	length = len(testInstance)-1
#last col:label
	for x in range(len(trainingSet)):
		dist = euclideanDistance(testInstance, trainingSet[x], length)
		distances.append([trainingSet[x], dist])
	distances.sort(key=operator.itemgetter(1))  #sort the first item in a list of distances list
	neighbors = []
	for x in range(k):
		neighbors.append(distances[x])  #append both trainingSet[x] and dist for k neighbors
	return neighbors

def getResponse(neighbors,proportion):
	classVotes = {}
	for x in range(len(neighbors)):
		response = neighbors[x][0][-1]
#trainingSet, the rightmost
#-1: from the right
		if response in classVotes:
			classVotes[response] += (1/neighbors[x][1])*proportion

		else:
			classVotes[response] = 0+ (1/neighbors[x][1])*proportion
#the weight of every one vote is proportional to distance
	sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
	return sortedVotes
#return the first response in case of a tie

def main():
	# prepare data
	trainingSet=[]
	testSet=[]
	NumOfTest = 100
	loadDataset('wisc_bc_data_preprocessed.csv', NumOfTest, trainingSet, testSet)
	print ('Number of train cases: ' + repr(len(trainingSet)) )
	print ('Number of test cases: ' + repr(len(testSet)) )
	# generate predictions
	predictions=[]
	k = 10
	for x in range(len(testSet)):
		neighbors = getNeighbors(trainingSet, testSet[x], k)
		proportion=100000
		result = getResponse(neighbors,proportion)[0][0]
		predictions.append(result)
		#print('predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))
	accuracy = getAccuracy(testSet, predictions)
	print('Accuracy: ' + repr(accuracy) + '%')
	
main()

Number of train cases: 469
Number of test cases: 100
Accuracy: 96.0%


The experiments gives a highest accuracy of around 96%:

Number of train cases: 469
Number of test cases: 100
Accuracy: 96.0%

## Part 2.2 adding a threshold to drop points too far

In the this experiment, weights are added according to distance and a threshold is added. The weight added is inversely proportional to distance, i.e. 1/distance.
Assume k=10, which apparently gives a higher accuracy in part 1
In case of a tie, the first response is returned.

In [17]:
def getResponse(neighbors,proportion):
	mean=0
	total=0
	for x in range(len(neighbors)):
		total+=neighbors[x][1]
	threshold=total/len(neighbors)*0.35
#threshold equals the x% of mean dist
	for x in range(len(neighbors)):
		if neighbors[x][1]>threshold:
			neighbors[x][1]=100000000
#if dist>threshold, dist is changed to a large number and negligible
	classVotes = {}
	for x in range(len(neighbors)):
		response = neighbors[x][0][-1]
#trainingSet, the rightmost
#-1: from the right
     
		if response in classVotes:

				classVotes[response] += (1/neighbors[x][1])*proportion
			#print(neighbors[x][1])
		else:

				classVotes[response] = 0+ (1/neighbors[x][1])*proportion
#the weight of every one vote is proportional to distance
	sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
	return sortedVotes
#return the first response in case of a tie

def main():
	# prepare data
	trainingSet=[]
	testSet=[]
	NumOfTest = 100
	loadDataset('wisc_bc_data_preprocessed.csv', NumOfTest, trainingSet, testSet)
	print ('Number of train cases: ' + repr(len(trainingSet)) )
	print ('Number of test cases: ' + repr(len(testSet)) )
	# generate predictions
	predictions=[]
	ListOfDistances=[]
	k = 10
	for x in range(len(testSet)):
		neighbors = getNeighbors(trainingSet, testSet[x], k)
		proportion=100
		result = getResponse(neighbors,proportion)[0][0]
		predictions.append(result)
		#print('predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))
	accuracy = getAccuracy(testSet, predictions)
	print('Accuracy: ' + repr(accuracy) + '%')
	
main()

Number of train cases: 469
Number of test cases: 100
Accuracy: 98.0%


When the threshold is 35% of mean:

Number of train cases: 469 Number of test cases: 100 Accuracy: 97.0%

When the threshold is 15%, 20%, 25% or 30% of mean:

Number of train cases: 469
Number of test cases: 100
Accuracy: 98.0%

When the threshold is 10% of mean:

Number of train cases: 469
Number of test cases: 100
Accuracy: 96.0%

## Part 2.3: in case of a tie, increase/decrease k

In [18]:
def getResponse(neighbors,proportion):
	mean=0
	total=0
	for x in range(len(neighbors)):
		total+=neighbors[x][1]
	threshold=total/len(neighbors)*0.35
#threshold equals the x% of mean dist
	for x in range(len(neighbors)):
		if neighbors[x][1]>threshold:
			neighbors[x][1]=100000000
#if dist>threshold, dist is changed to a large number and negligible
	classVotes = {}
	for x in range(len(neighbors)):
		response = neighbors[x][0][-1]
#trainingSet, the rightmost
#-1: from the right
     
		if response in classVotes:

				classVotes[response] += (1/neighbors[x][1])*proportion
			#print(neighbors[x][1])
		else:

				classVotes[response] = 0+ (1/neighbors[x][1])*proportion
#the weight of every one vote is proportional to distance
	sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
	return sortedVotes
#in case of a tie, return both

def main():
	# prepare data
	trainingSet=[]
	testSet=[]
	NumOfTest = 100
	loadDataset('wisc_bc_data_preprocessed.csv', NumOfTest, trainingSet, testSet)
	print ('Number of train cases: ' + repr(len(trainingSet)) )
	print ('Number of test cases: ' + repr(len(testSet)) )
	# generate predictions
	predictions=[]
	k = 10
	for x in range(len(testSet)):
		neighbors = getNeighbors(trainingSet, testSet[x], k)
		proportion=1
		if len(getResponse(neighbors,proportion))==2:
			while getResponse(neighbors,proportion)[0][0]==getResponse(neighbors,proportion)[1][0]:
				print("tie")
				k+=1
#votes of 1st and 2st in the sorted list are the same
				neighbors=getNeighbors(trainingSet,testSet[x],k)
		result = getResponse(neighbors,proportion)[0][0]
		predictions.append(result)
		#print('predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))
	accuracy = getAccuracy(testSet, predictions)
	print('Accuracy: ' + repr(accuracy) + '%')
	
main()

Number of train cases: 469
Number of test cases: 100
Accuracy: 93.0%


In case weights are added according to distance, apparently a tie almost does not happen.

In the following experiment,
threshold: false
weights according to distance: false
variation of k in case of tie: true
k: 10

In [19]:
def getResponse(neighbors,proportion):
	classVotes = {}
	for x in range(len(neighbors)):
		response = neighbors[x][0][-1]
#trainingSet, the rightmost
#-1: from the right
		if response in classVotes:
			classVotes[response] += 1
			#print(neighbors[x][1])
		else:
			classVotes[response] = 1

	sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
	#print(sortedVotes)
	return sortedVotes
#in case of a tie, return both

def main():
	# prepare data
	trainingSet=[]
	testSet=[]
	NumOfTest = 100
	loadDataset('wisc_bc_data_preprocessed.csv', NumOfTest, trainingSet, testSet)
	print ('Number of train cases: ' + repr(len(trainingSet)) )
	print ('Number of test cases: ' + repr(len(testSet)) )
	# generate predictions
	predictions=[]
	k = 10
	for x in range(len(testSet)):
		neighbors = getNeighbors(trainingSet, testSet[x], k)
		proportion=1
		if len(getResponse(neighbors,proportion))==2:
			while getResponse(neighbors,proportion)[0][0]==getResponse(neighbors,proportion)[1][0]:
				print("tie")
				k+=1
#votes of 1st and 2st in the sorted list are the same
				neighbors=getNeighbors(trainingSet,testSet[x],k)
		result = getResponse(neighbors,proportion)[0][0]
		predictions.append(result)
		#print('predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))
	accuracy = getAccuracy(testSet, predictions)
	print('Accuracy: ' + repr(accuracy) + '%')
	
main()

Number of train cases: 469
Number of test cases: 100
Accuracy: 95.0%


The best accuracy gives 99%:

Number of train cases: 469
Number of test cases: 100
Accuracy: 99.0%

In case every nearest neighbor has one vote irespect to distance, apparently a tie almost does not happen neither.
So it is assumed the case of a tie is negligible.

# Part 3 Different ways of splitting the data

In the above the ratio of training subset and testing subset is fixed to be 469:100 and the test cases are selected randomly.
Lastly, the ratio will be varied, using setting of the best performance attainable so far:

threshold: false
weights according to distance: false
variation of k in case of tie: true
k: 10

In [21]:
def loadDataset(filename, ratio, trainingSet=[] , testSet=[]):
	with open(filename, 'r') as csvfile:
	    lines = csv.reader(csvfile)
	    next(lines, None)
#skip the first line which is headers
	    datarow = list(lines)
	    for x in range(len(datarow)):
	        for y in range(30):
	            datarow[x][y] = float(datarow[x][y])
	        
	        if random.random() < ratio:
#0<=random.random()<=1, ratio is train case:test case
	            trainingSet.append(datarow[x])
	        else:
	            testSet.append(datarow[x])

def getResponse(neighbors,proportion):
	classVotes = {}
	for x in range(len(neighbors)):
		response = neighbors[x][0][-1]
#trainingSet, the rightmost
#-1: from the right
		if response in classVotes:
			classVotes[response] += 1
			#print(neighbors[x][1])
		else:
			classVotes[response] = 1

	sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
	#print(sortedVotes)
	return sortedVotes
#in case of a tie, return both

def main():
	# prepare data
	trainingSet=[]
	testSet=[]
	ratio = 0.6
	loadDataset('wisc_bc_data_preprocessed.csv', ratio, trainingSet, testSet)
	print ('Number of train cases: ' + repr(len(trainingSet)) )
	print ('Number of test cases: ' + repr(len(testSet)) )
	# generate predictions
	predictions=[]
	k = 10
	for x in range(len(testSet)):
		neighbors = getNeighbors(trainingSet, testSet[x], k)
		proportion=1
		if len(getResponse(neighbors,proportion))==2:
			while getResponse(neighbors,proportion)[0][0]==getResponse(neighbors,proportion)[1][0]:
				print("tie")
				k+=1
#votes of 1st and 2st in the sorted list are the same
				neighbors=getNeighbors(trainingSet,testSet[x],k)
		result = getResponse(neighbors,proportion)[0][0]
		predictions.append(result)
		#print('predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))
	accuracy = getAccuracy(testSet, predictions)
	print('Accuracy: ' + repr(accuracy) + '%')
	
main()

Number of train cases: 327
Number of test cases: 242
Accuracy: 95.86776859504133%


In case ratio is 0.8 or 0.9, the best case is:

Number of train cases: 456
Number of test cases: 113
Accuracy: 100.0%

Number of train cases: 514
Number of test cases: 55
Accuracy: 100.0%

Number of train cases: 507
Number of test cases: 62
Accuracy: 100.0%

If the ratio is 0.7, the highest accuracy becomes:

Number of train cases: 417
Number of test cases: 152
Accuracy: 98.68421052631578%

When the ratio is 0.6, the highest accuracy is:

Number of train cases: 337
Number of test cases: 232
Accuracy: 97.41379310344827%

It shows that the higher the number of train cases, the more accurate the prediction is perhaps due to a larger knowledge base to assist in a judgement. The less the number of test cases, the higher chance to reach the highest accuracy because of less challenges probably.

# Conclusions
In conclusion, the best result setting among the above experiments are:
ratio of splitting: 0.9
threshold: false
weights according to distance: false
variation of k in case of tie: true
k: 10

# References
1. https://machinelearningmastery.com/tutorial-to-implement-k-nearest-neighbors-in-python-from-scratch/
2. https://www.python-course.eu/k_nearest_neighbor_classifier.php